In [1]:
#pip install -q git+https://github.com/tensorflow/docs

In [2]:
#!pip install --upgrade setuptools

In [3]:
#!pip install --upgrade protobuf

In [4]:
#pip install torch torchvision torchaudio

In [5]:
#!pip install nltk

In [6]:
import pandas as pd
import torch
import tensorflow as tf
from rouge import Rouge

def chunk_text(text, chunk_size=1024):
    tokens = text.split()
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    return [" ".join(chunk) for chunk in chunks]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
def summarize_text(text):
    device = torch.device("cuda")
    print(device)
    tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")  
    model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
    input_tokenized = tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=150,
                                  max_length=250,
                                  early_stopping=True)
    return [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

# Load the summaries.csv file into a dataframe
df = pd.read_csv("new_summaries.csv",)

# Summarize the text in each row of the 'Text' column
for index, row in df.iterrows():
    text = row["Text"]
    chunks = chunk_text(text)
    final_summary = ""
    for chunk in chunks:
        summary = summarize_text(chunk)
        final_summary += summary
    df.at[index, "Predicted Summary"] = final_summary

# Calculate the Rouge scores for each row9
for index, row in df.iterrows():
    rouge = Rouge()
    original_summary = row["Original Summary"]
    predicted_summary = row["Predicted Summary"]
    scores = rouge.get_scores(predicted_summary, original_summary)
    rouge_1_f1 = scores[0]['rouge-1']['f']
    rouge_2_f1 = scores[0]['rouge-2']['f']
    rouge_l_f1 = scores[0]['rouge-l']['f']
    df.at[index, 'R1'] = rouge_1_f1
    df.at[index, 'R2'] = rouge_2_f1
    df.at[index, 'Rl'] = rouge_l_f1
df.to_csv('Output_DistilBart_chunking_with_100doc.csv', index=False)

cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda


#  Rouge Score

In [7]:
# Calculate the average scores
r1_avg = df['R1'].mean()
r2_avg = df['R2'].mean()
rl_avg = df['Rl'].mean()

#Print the average scores
print(f"Average R1 Score: {r1_avg}")
print(f"Average R2 Score: {r2_avg}")
print(f"Average Rl Score: {rl_avg}")
df.to_csv('Output_DistilBart_chunking_with_100doc.csv', index=False)

Average R1 Score: 0.3475501350047469
Average R2 Score: 0.17540581038020558
Average Rl Score: 0.32156218550216403


# Precision, Recall and F1 Score

In [8]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.metrics import precision, recall, f_measure

df = pd.read_csv('Output_DistilBart_chunking_with_100doc.csv')

for index, row in df.iterrows():
    predicted_summary = row['Predicted Summary']
    original_summary = row['Original Summary']
    
    predicted_tokens = set(word_tokenize(predicted_summary))
    original_tokens = set(word_tokenize(original_summary))

    precision_score = precision(original_tokens, predicted_tokens)
    recall_score = recall(original_tokens, predicted_tokens)
    f1_score = f_measure(original_tokens, predicted_tokens)
    
    df.at[index, 'Precision'] = precision_score
    df.at[index, 'Recall'] = recall_score
    df.at[index, 'F1 Score'] = f1_score

df.to_csv('Output_DistilBart_chunking_with_100doc.csv', index=False)


In [9]:
import pandas as pd

df = pd.read_csv('Output_DistilBart_chunking_with_100doc.csv')

precision_mean = df['Precision'].mean()
recall_mean = df['Recall'].mean()
f1_score_mean = df['F1 Score'].mean()

print("Precision mean:", precision_mean)
print("Recall mean:", recall_mean)
print("F1 Score mean:", f1_score_mean)


Precision mean: 0.38710931155705963
Recall mean: 0.4482668759057593
F1 Score mean: 0.3776976592809651


# BLEU Score

In [10]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu

df = pd.read_csv('Output_DistilBart_chunking_with_100doc.csv')

for index, row in df.iterrows():
    predicted_summary = row['Predicted Summary']
    original_summary = row['Original Summary']
    
    predicted_tokens = word_tokenize(predicted_summary)
    original_tokens = word_tokenize(original_summary)

    bleu_score = sentence_bleu([original_tokens], predicted_tokens)
    
    df.at[index, 'BLEU Score'] = bleu_score

df.to_csv('Output_DistilBart_chunking_with_100doc.csv', index=False)


import pandas as pd

df = pd.read_csv('Output_DistilBart_chunking_with_100doc.csv')

bleu_score_mean = df['BLEU Score'].mean()

print("BLEU Score mean:", bleu_score_mean)


C:\Users\Pranav\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU Score mean: 0.11039003958433363
